In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import Library
import glob
import re
import os
from scipy import io, stats

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD, Adam


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
folders = glob.glob('/content/drive/MyDrive/Project_B_Data/EPOT_Data/*')

In [4]:
len(folders)

244

In [5]:
final = np.empty([1,7])

for folder in folders:
  data = io.loadmat(folder)
  #print(data.values())

  arr = list(data.values())
  #print(arr[-1])

  final_data = arr[-1]
  final_data = np.asarray(final_data)
  #print(final_data)                               # printing the final data
  #print(folder)
  name = re.split('/',folder)
  #print(name)
  exp_name = name[-1]
  #print(exp_name)

  file_list = [exp_name]
  for fname in file_list:
    remove_extension = os.path.splitext(exp_name)
    #print(remove_extension)
    naam_file = re.split('_',remove_extension[-2])
    #print(naam_file)
    x_coor = naam_file[-2]
    #print(x_coor)
    y_coor = naam_file[-1]
    #print(y_coor)

    #print('X-Coordinate is: '+x_coor+' Y-Coordinate is: '+y_coor)

    x = np.full((len(final_data),1), x_coor)
    y = np.full((len(final_data),1), y_coor)


    all_data = np.c_[x, y, final_data]
    #print(all_data)

  final = np.vstack([final, all_data])
  

print(final)



[['4.66823449024257e-310' '8.4879831693e-313' '2.355415328475e-312' ...
  '2.4190752022e-312' '8.70018274864e-313' '6.087e-320']
 ['250' '295' '0.0' ... '0.0' '0.0' '0.0']
 ['250' '295' '1e-08' ... '0.0' '0.0' '0.0']
 ...
 ['325' '295' '0.00019997' ... '1.30776' '-0.505441' '-0.98446']
 ['325' '295' '0.00019998' ... '1.3072' '-0.506401' '-0.984543']
 ['325' '295' '0.00019999' ... '1.30663' '-0.507363' '-0.984628']]


In [6]:
final.shape

(4880001, 7)

In [7]:
final = np.delete(final, (0), axis=0)
print(final)

[['250' '295' '0.0' ... '0.0' '0.0' '0.0']
 ['250' '295' '1e-08' ... '0.0' '0.0' '0.0']
 ['250' '295' '2e-08' ... '0.0' '0.0' '0.0']
 ...
 ['325' '295' '0.00019997' ... '1.30776' '-0.505441' '-0.98446']
 ['325' '295' '0.00019998' ... '1.3072' '-0.506401' '-0.984543']
 ['325' '295' '0.00019999' ... '1.30663' '-0.507363' '-0.984628']]


In [8]:
final.shape

(4880000, 7)

In [9]:
final.dtype

dtype('<U32')

In [10]:
final = final.astype(float)

In [11]:
#Data Standardisation

x_data = final[:, 3:8]
y_data = final[: , 0:2]

In [12]:
scaler = StandardScaler()

X, y = scaler.fit_transform(x_data), y_data
X.shape


(4880000, 4)

In [13]:
###Frame Preparation

Fs = 50

frame_size = Fs*4
hop_size = Fs*2

In [14]:
def get_frames(data, frame_size, hop_size):

    N_FEATURES = 4

    frames = []
    labels = []
    for i in range(0, len(data) - frame_size, hop_size):
        x1 = final[:,3][i: i + frame_size]
        x2 = final[:,4][i: i + frame_size]
        x3 = final[:,5][i: i + frame_size]
        x4 = final[:,6][i: i + frame_size]
        
        # Retrieve the most often used label in this segment
        label = stats.mode(final[:,0:2][i: i + frame_size])[0][0]
        frames.append([x1, x2, x3, x4])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

In [15]:
X, y = get_frames(X, frame_size, hop_size)
X. shape, y.shape

((48798, 200, 4), (48798, 2))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train[0].shape

(39038, 200, 4)
(9760, 200, 4)
(39038, 2)
(9760, 2)


(200, 4)

In [17]:
#X_train = X_train.reshape(39038, 200, 4, 1)
#X_test = X_test.reshape(9760, 200, 4, 1)

In [18]:
#X_train[0].shape, X_test[0].shape

In [21]:
from tensorflow.python.keras import layers
#### Build model

model = keras.Sequential([
                          keras.layers.Flatten(input_shape=X_train[0].shape),           ###Input Layer
                          keras.layers.Dense(400, activation='relu', kernel_regularizer=keras.regularizers.l1(0.001)),          ##Hidden Layer 1
                          keras.layers.Dense(200, activation='relu', kernel_regularizer=keras.regularizers.l1(0.001)),           ###Hidden Layer 2
                          keras.layers.Dense(100, activation='relu'),           ##Hidden Layer 3
                          keras.layers.Dense(2, activation='softmax')         ##Output Layer
])

####Model Compile
model.compile(optimizer= Adam(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

#####Training the model with epochs 10
model.fit(X_train, y_train, epochs=10)

test_loss, test_acc = model.evaluate(X_test, y_test)

print('Tested Acc: ', test_acc)

Epoch 1/10
1220/1220 [==============================] - 9s 7ms/step - loss: 62809399873241088.0000 - accuracy: 0.5585
Epoch 2/10
1220/1220 [==============================] - 8s 7ms/step - loss: 1445280134603997184.0000 - accuracy: 0.5585
Epoch 3/10
1220/1220 [==============================] - 8s 7ms/step - loss: 6404482058713825280.0000 - accuracy: 0.5585
Epoch 4/10
1220/1220 [==============================] - 8s 7ms/step - loss: 18539333947884568576.0000 - accuracy: 0.5585
Epoch 5/10
1220/1220 [==============================] - 9s 7ms/step - loss: 40222664220789964800.0000 - accuracy: 0.5585
Epoch 6/10
1220/1220 [==============================] - 8s 7ms/step - loss: 85064200868006461440.0000 - accuracy: 0.5585
Epoch 7/10
1220/1220 [==============================] - 8s 7ms/step - loss: 141206795102434885632.0000 - accuracy: 0.5585
Epoch 8/10
1220/1220 [==============================] - 9s 7ms/step - loss: 670401829511953907712.0000 - accuracy: 0.4630
Epoch 9/10
1220/1220 [=============